In [1]:
from wordlist import WordList
from learner import MyLearner
import json

In [2]:
a = WordList()

In [ ]:
from IPython.display import Markdown

# Beautiful printing of the text
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

to_markdown(chat_completion.candidates[0].content.parts[0].text)

In [ ]:
# prepare the llm model
import glob
import os
import re
import pathlib
import time
import textwrap
import google.generativeai as genai

# read the api_key from a text file
with open("var.txt","r") as f:
    var = f.read()
# connect to gemini model    
genai.configure(api_key=var)
model = genai.GenerativeModel('models/gemini-2.0-flash')

In [ ]:
# read ingestion list from a text file
with open('input3.txt', 'r') as f:
    data = f.read()


In [ ]:
# get a verified list from gemini
prompt_text = """
Here is a list of dutch words and short phrases. Some of them are translated to english and russian, some only to english. 
Add missing translations of dutch words to english or to russian columns and return it in json format. Make sure that there 
are no missing or null translations in english or in russian. The json format should be the following:
{'althans': {'english': 'at least', 'russian': 'по крайней мере'},
'overleggen': {'english': 'to discuss', 'russian': 'обсуждать'},
'het hoorcollege': {'english': 'lecture', 'russian': 'лекция'},
}
The words: """ 
chat_completion = model.generate_content(prompt_text + data)



In [ ]:
# correct the list and convert it to json and then to 2d list

import json
raw_string = chat_completion.candidates[0].content.parts[0].text

# 1. Clean up the Markdown code block wrappers
clean_json = raw_string.strip().removeprefix('```json').removesuffix('```').strip()

# 2. Parse it with json.loads()
data = json.loads(clean_json)

# 3. Convert it to a 2D lis
words_list = []
for dutch_word in data:
    dutch = dutch_word
    try:
      english = data[dutch_word]['english']
    except KeyError:
       english = ''
    try: 
      russian = data[dutch_word]['russian']
    except KeyError:
       russian = ''
    words_list.append([dutch, english, russian]) 



In [ ]:
# ingest the list to the list
a.ingest_list(words_list=words_list)

In [ ]:
from difflib import SequenceMatcher

def get_similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

similarity = get_similarity("het geweld", "de geweld")  # maybe 0.66
similarity

In [8]:
from wordlist import WordList
import random
import numpy as np
class MyLearner:
    def __init__(self, words_list:WordList):
        self.status = 0
        self.words_list = words_list
        self.all_words= self.words_list.get_all_words()
        self.n_list = len(self.words_list.get_all_words())
        self.n_to_learn = 0
        self.threshold0 = 0.3
        self.threshold1 = 0.7
        # Portion of words in each group
        self.partition = [0.6, 0.3, 0.1]
        self.ind0_sampled = [] # list of indeces of words with low score to be learned
        self.ind1_sampled = [] # list of indeces of words with medium score to be learned
        self.ind2_sampled = [] # list of indeces of words with high score to be learned
        self.ind_sampled = [] # list of indeces of words with all score to be learned
        self.task_list_with_options = []
        self.score_list = [] # score list

    def SelectWords(self,n:int=10):
        # n, int, number of words to test
        self.n_to_learn = n
        self.score_list = [item['score'] for item in self.all_words]
        sorted_indices = np.argsort(self.score_list)
        print("Sorted indices:", sorted_indices)  # [1, 3, 0, 2]
        # ind0 = [x for x in score_list if x < self.threshold0]
        # get indeces of the words with different scores
        ind0 = []
        ind1 = []
        ind2 = []
        for i_x,x in enumerate(self.score_list):
            if x < self.threshold0:
                ind0.append(i_x)
            elif (x >= self.threshold0 and x < self.threshold1):
                ind1.append(i_x)
            else:
                ind2.append(i_x)
        ind_len = [len(ind0), len(ind1), len(ind2)]
        print(f"ind_len={ind_len}")
        # if we have enough words in the list, lets try to take a random
        # number of words with certain score
        if self.n_list >= self.n_to_learn:
            n0 = self.n_to_learn
            n1 = 0
            n2 = 0
            if len(ind1) > 0 and len(ind2) == 0:
                n0 = round(self.n_to_learn*self.partition[0])
                n1 = self.n_to_learn - n0
                n2 = 0
            elif len(ind1) > 0 and len(ind2) > 0:
                n0 = round(self.n_to_learn*self.partition[0])
                n1 = round(self.n_to_learn*self.partition[1])
                n2 = self.n_to_learn - (n1 + n0)
            print(n0,n1,n2)
            if ind_len[0] >= n0:
                self.ind0_sampled = random.sample(ind0,n0)
            else:
                self.ind0_sampled = random.sample(ind0,ind_len[0])
            if ind_len[1] >= n1:
                self.ind1_sampled = random.sample(ind1,n1)
            else:
                self.ind1_sampled = random.sample(ind1,ind_len[1])
            if ind_len[2] >= n2:
                self.ind2_sampled = random.sample(ind2,n2)
            else:
                self.ind2_sampled = random.sample(ind2,ind_len[2])
            print(self.ind0_sampled, self.ind1_sampled, self.ind2_sampled)
        else:
            print(f"""Error: Requested too many words (n={self.n_to_learn}). 
                  List contains onlygit {self.n_list} words.""")
    def CreateModel(self):
        # prepare the llm model
        import google.generativeai as genai

        # read the api_key from a text file
        with open("var.txt","r") as f:
            var = f.read()
        # connect to gemini model    
        genai.configure(api_key=var)
        self.model = genai.GenerativeModel('models/gemini-2.0-flash')
        
    
    def GenerateTask(self):
        tasks_list= []
        n = 3 # number of options to choose from
        word_learn0 = [d.all_words[i]['word'] for i in d.ind0_sampled]
        word_learn1 = [d.all_words[i]['word'] for i in d.ind1_sampled]
        word_learn2 = [d.all_words[i]['word'] for i in d.ind2_sampled]
        trans_learn0 = [d.all_words[i]['translation'] for i in d.ind0_sampled]
        trans_learn1 = [d.all_words[i]['translation'] for i in d.ind1_sampled]
        trans_learn2 = [d.all_words[i]['translation'] for i in d.ind2_sampled]
        self.ind_sampled = d.ind0_sampled + d.ind1_sampled + d.ind2_sampled # combined
        # get a verified list from gemini
        prompt_text = f"""
        Here is a list of {self.n_to_learn} words I want to learn in a form of quiz. After the words there is a list of their translations. 
        Return only a 2d list of size {self.n_to_learn}x{n} in json format, where for each translation propose {n} similar options to choose from in a quize. Don't return any extra text.
        Expected return format: [['word1', 'right_translation1','option1_1','option1_2','option1_3'],['word2', 'right_translation2','option2_1','option2_2','option2_3'],...]
        The words and translations: """ 
        print(word_learn0)

        chat_completion = self.model.generate_content(prompt_text +
                                                           ", ".join(word_learn0)+
                                                           ", ".join(word_learn1)+
                                                           ", ".join(word_learn2)+ 
                                                           " Translations: " + 
                                                           ", ".join(trans_learn0) + 
                                                           ", ".join(trans_learn1) + 
                                                           ", ".join(trans_learn2))
        raw_string = chat_completion.candidates[0].content.parts[0].text
        # 1. Clean up the Markdown code block wrappers
        clean_json = raw_string.strip().removeprefix('```json').removesuffix('```').strip()
        # 2. Parse it with json.loads()
        self.task_list_with_options = json.loads(clean_json)
        print(self.task_list_with_options)

    def DoTask(self, if_genererate:bool=False, n_words:int = 5):
        if if_genererate:
            self.SelectWords(n_words)
            self.GenerateTask()
        for i in range(self.n_to_learn):
            print(f"Task {i+1}:")
            print(f"Nederlands: {self.task_list_with_options[i][0]} ")
            abcd = random.sample([1,2,3,4],4)
            print(f"A. {self.task_list_with_options[i][abcd[0]]} ")
            print(f"B. {self.task_list_with_options[i][abcd[1]]} ")
            print(f"C. {self.task_list_with_options[i][abcd[2]]} ")
            print(f"D. {self.task_list_with_options[i][abcd[3]]} ")
            answer = input("Answer: ")
            if (answer == "A" or answer == "a") and abcd[0] == 1:
                print("Correct!")
                new_score = self.score_list[i]+0.1
            elif (answer == "B" or answer == "b") and abcd[1] == 1:
                print("Correct!")
                new_score = self.score_list[i]+0.1
            elif (answer == "C" or answer == "c") and abcd[2] == 1:
                print("Correct!")
                new_score = self.score_list[i]+0.1
            elif (answer == "D" or answer == "d") and abcd[3] == 1:
                print("Correct!")
                new_score = self.score_list[i]+0.1
            else:
                print("Wrong :(")
                new_score = self.score_list[i]-0.1
            if new_score > 1.0:
                new_score = 1.0
            elif new_score < 0.0:
                new_score = 0.0
            self.words_list.update_score(self.ind_sampled[i],new_score)

# from learner import MyLearner
d = MyLearner(a)
d.SelectWords(n=5)  
d.CreateModel()
d.GenerateTask()


Sorted indices: [ 0  1  2  4  6  5  9  8 13 14 10 11 15 19 18 17 28 25 26 20 22 21 23 24
 30 29 27 38 36 37 34 33 56 57 58 51 52 39 41 40 46 45 43 44 47 50 49 48
 61 62 63 59 54 53 55 64 31 16  7 12 42 32 35  3 60]
ind_len=[65, 0, 0]
5 0 0
[26, 3, 54, 52, 4] [] []
['solliciteren naar', 'het geruis', 'loeien', 'versleten', 'vergissen']
[['solliciteren naar', 'to apply for a job', 'to look for', 'to work', 'to create'], ['het geruis', 'rustling', 'silence', 'thunder', 'whispering'], ['loeien', 'to moo', 'to bark', 'to roar', 'to meow'], ['versleten', 'worn out', 'new', 'repaired', 'clean'], ['vergissen', 'to be mistaken', 'to be correct', 'to understand', 'to agree']]


In [6]:
d.DoTask(if_genererate=True,n_words=5)

Sorted indices: [ 0  1  2  3  4  5  6  8 12  9 10 11 14 13 15 17 28 18 19 20 22 21 23 24
 30 25 26 27 29 37 34 33 56 57 50 36 38 39 41 40 44 45 42 43 46 47 49 48
 61 62 58 51 52 53 54 55 63 59 64  7 31 32 35 16 60]
ind_len=[65, 0, 0]
5 0 0
[25, 13, 43, 4, 8] [] []
['richten op', 'huiveren', 'schuiven', 'vergissen', 'het tikken']
[['richten op', 'to aim at, to focus on', 'to avoid', 'to ignore', 'to deny'], ['huiveren', 'to shiver', 'to sweat', 'to jump', 'to sing'], ['schuiven', 'to slide', 'to stick', 'to rotate', 'to dig'], ['vergissen', 'to be mistaken', 'to understand', 'to agree', 'to know'], ['het tikken', 'ticking', 'ringing', 'buzzing', 'humming']]
Task 1:
Nederlands: richten op 
A. to deny 
B. to avoid 
C. to aim at, to focus on 
D. to ignore 
Wrong :(
Task 2:
Nederlands: huiveren 
A. to jump 
B. to sweat 
C. to sing 
D. to shiver 
Correct!
Task 3:
Nederlands: schuiven 
A. to slide 
B. to dig 
C. to rotate 
D. to stick 
Correct!
Task 4:
Nederlands: vergissen 
A. to understand 

In [7]:
d.words_list.get_all_words()

[{'id': 1,
  'word': 'fel',
  'translation': 'bright',
  'translation2': 'яркий',
  'score': 0.0},
 {'id': 2,
  'word': 'telkens',
  'translation': 'each time, every time, again and again',
  'translation2': 'каждый раз, снова и снова',
  'score': 0.0},
 {'id': 3,
  'word': 'het suizen',
  'translation': 'whistling',
  'translation2': 'свист, шум, шипение',
  'score': 0.0},
 {'id': 4,
  'word': 'het geruis',
  'translation': 'rustling',
  'translation2': 'шуршание, шелест',
  'score': 0.1},
 {'id': 5,
  'word': 'vergissen',
  'translation': 'to be mistaken',
  'translation2': 'ошибаться',
  'score': 0.0},
 {'id': 6,
  'word': 'het zuiden',
  'translation': 'south',
  'translation2': 'юг',
  'score': 0.0},
 {'id': 7,
  'word': 'bewijzen',
  'translation': 'to prove',
  'translation2': 'доказывать',
  'score': 0.0},
 {'id': 8,
  'word': 'de nagel',
  'translation': 'nail',
  'translation2': 'ноготь',
  'score': 0.1},
 {'id': 9,
  'word': 'het tikken',
  'translation': 'ticking',
  'trans

In [ ]:
import random

q = [random.random() for _ in range(10)]
b = random.sample(q,10)
sum(q)